# Monte Carlo Simulation of Water

In this exercise we will use the Metropolis-Hastings Monte Carlo algorithm to study a collection of water molecules at a given temperature. As you probably know, water can be found in a number of phases - gas, liquid, ice - and its complex properties are strongly affected by water's ability to form hydrogen bonds. Here we approximate water using a so-called three-point model where oxygen and the two hydrogens are simply treated a three sites, interacting via a classical (and of course approximate) potential energy function.

To simulate the system, we will use the [_Faunus_ software](https://github.com/mlund/faunus).
To learn more about the input and output, see the [online manual](https://faunus.readthedocs.io).

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import nglview as nv
import mdtraj as md
import json

## Task: Get familiar with the input and output

The simulation is controlled by the input file `water.yml` and the shell command below sends it to the Faunus program. Get familiar with the input (`water.yml`) and output (`out.json`):

1. What Monte Carlo moves do we have?
1. What does _acceptance_ in the output mean?
1. Plot `energy.dat` and describe how the system energy develops with MC steps?
1. Things to vary and play with:
  - number of particles (what is the density in g/l?)
  - atomic charges (what is the dipole moment of water? _Debye_ is a common unit)). You could try to replace water with H2S and see what happens in the _NPT_ ensemble.
  - number of MC steps
1. How does the _acceptance_ vary with the so-called _displacement parameter_?
1. Plot the radial distribution function, $g(r)$. See `rdf.out`
  - how does it vary with density and/or dipole moment?
  
  
_Tip - restarting simulations_: When starting a simulation, the molecules are place at random positions and orientations. At the end of the simulation, a file `state.json` will be saved, and you can use it as the starting point for a new simulation by adding `-s state.json` after `faunus` shell command.


In [ ]:
!yason.py water.yml | faunus # run simulation!

## Task: Visualize simulation trajectory

During simulation, the configurations are periodically saved to disk and you can control this in the `analysis` section. Here we use the module [`nglview`](https://github.com/nglviewer/nglview) to visualise this directly in the notebook. Consult the nglview documentation and try to customise the molecular representation. Describe what happens to the water molecules during the simulation.

In [ ]:
traj = md.load('traj.xtc', top='confout.gro')
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_spacefill(selection='*')
view.background = 'white'
view.camera = 'orthographic'
view.center()
view

## Task: Add a Monte Carlo move to change the _volume_ of the simulation container

In the previous section we used a _fixed_ simulation container and thus simulated in the _canonical ensemble_, i.e. with the number of particles, volume, and temperature constant ($NVT$). The Monte Carlo moves affected only the positions and orientations of the water molecules. We now keep the _pressure_ constant and now also perform moves on the _volume_ using the Metropolis algorithm. This now generate configurations in the _isobaric-isothermal ensemble_ ($PVT$).

Add the following to the energy and move sections of the input, respectively:

- `isobaric: {P/atm: 1.0}` # energy term
- `volume: {dV: 0.03, method: isotropic}` # move term

Consult the [Faunus manual](https://faunus.readthedocs.io) to describe the used parameters and how they affect the simulation. Things to explore:

- What density do you obtain
- How does the box change during simulation
- What happens to the density if you decrease or even remove the dipole moment of water?

## Tips and Tricks

Information and statistics about the simulation is stored in `out.json`. Python can easily read JSON files as shown below. The output file can be fairly nested, so it's always a good idea to first inspect it by double-clicking it in the file browser.

In [ ]:
with open('out.json') as file:
    d = json.load(file) # --> dict
    acceptance = d['moves'][0]['moltransrot']['acceptance']
    print('Acceptance probability for the first move is {:.1f}%'.format(acceptance*100))

#### Plotting

We can also plot data using `matplotlib`. By first inspecting the `energy.dat` file in the file browser, we see that it should be read like this:

In [ ]:
step, total, selfenergy, nonbonded = np.loadtxt('energy.dat', unpack=True, skiprows=1)
print('average total energy = {:.3E} kT'.format(total.mean()))

In [ ]:
plt.plot(step, total, label='total')
plt.plot(step, selfenergy, label='self energy')
plt.plot(step, nonbonded, label='nonbonded')
plt.xlabel('Monte Carlo steps')
plt.ylabel(r'Energy ($k_BT)$')
plt.title('Energy vs. steps')
plt.legend(loc=0, frameon=False)